In [ ]:
'''

整體功能描述

'''

In [1]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/素材" , static_folder = "./素材/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [2]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [3]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    # 讀取並轉換
    result_message_array =[]
    replyJsonPath = "素材/關注/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

    # 呼叫個人用戶名，與用戶第一次接觸打招呼
    user_profile = line_bot_api.get_profile(event.source.user_id)
    hello = {
      "type": "text",
      "text": "您好!" + str(user_profile.display_name) + "\n歡迎您使用「RunningAI」"
    }
    result_message_array.insert(0, TextSendMessage.new_from_json_dict(hello))
    
        
    #綁定 rich_menu
    linkRichMenuId = open("素材/rich_menu_4/rich_menu_id", 'r').read()
    line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
    
    
    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )

    #擷取tag訊息
    dn = "ID_message/user_event.txt"
    user_event = {}
    with open(dn, "a") as myfile:
        user_event["user_id"] = vars(user_profile)["user_id"]
        user_event["tag"] = "user of first entering"
        user_event["time"] = event.timestamp
        user_event["product_ID"] = "now to choose"
        myfile.write(json.dumps(user_event,sort_keys=True))
        myfile.write('\r\n')

In [ ]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    dict1 = {'路線推薦': '1-1', '活動賽事': '2-1', '歷史紀錄': '5-1', '聯絡我們': '6-1'}
    # 讀取本地檔案，並轉譯成消息
    result_message_array =[]
    replyJsonPath = "素材/"+event.message.text+"/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

    # 發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )


In [ ]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):
    


    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'folder' in query_string_dict:
    
        result_message_array =[]

        replyJsonPath = '素材/'+query_string_dict.get('folder')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'menu' in query_string_dict:
 
        linkRichMenuId = open("素材/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        replyJsonPath = '素材/'+query_string_dict.get('menu')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )

In [ ]:
# 5.歷史紀錄

In [4]:
jsonObject = {
  "type": "template",
  "altText": "this is a carousel template",
  "template": {
    "type": "carousel",
    "actions": [],
    "columns": [
      {
        "thumbnailImageUrl": "SPECIFY_YOUR_IMAGE_URL",
        "title": "運動時間",
        "text": "運動類型",
        "actions": [
          {
            "type": "message",
            "label": "查看詳細記錄",
            "text": "查看詳細記錄"
          }
        ]
      },
      {
        "thumbnailImageUrl": "SPECIFY_YOUR_IMAGE_URL",
        "title": "運動時間",
        "text": "運動類型",
        "actions": [
          {
            "type": "message",
            "label": "查看詳細記錄",
            "text": "查看詳細記錄"
          }
        ]
      },
      {
        "thumbnailImageUrl": "SPECIFY_YOUR_IMAGE_URL",
        "title": "運動時間",
        "text": "運動類型",
        "actions": [
          {
            "type": "message",
            "label": "查看詳細記錄",
            "text": "查看詳細記錄"
          }
        ]
      }
    ]
  }
}

In [5]:
#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)
#引入按鍵模板
from linebot.models.template import(
    CarouselTemplate
)

carousel_template_message = TemplateSendMessage.new_from_json_dict(jsonObject)
print(carousel_template_message)#引入所需要的消息與模板消息

{"altText": "this is a carousel template", "template": {"columns": [{"actions": [{"label": "\u67e5\u770b\u8a73\u7d30\u8a18\u9304", "text": "\u67e5\u770b\u8a73\u7d30\u8a18\u9304", "type": "message"}], "text": "\u904b\u52d5\u985e\u578b", "thumbnailImageUrl": "SPECIFY_YOUR_IMAGE_URL", "title": "\u904b\u52d5\u6642\u9593"}, {"actions": [{"label": "\u67e5\u770b\u8a73\u7d30\u8a18\u9304", "text": "\u67e5\u770b\u8a73\u7d30\u8a18\u9304", "type": "message"}], "text": "\u904b\u52d5\u985e\u578b", "thumbnailImageUrl": "SPECIFY_YOUR_IMAGE_URL", "title": "\u904b\u52d5\u6642\u9593"}, {"actions": [{"label": "\u67e5\u770b\u8a73\u7d30\u8a18\u9304", "text": "\u67e5\u770b\u8a73\u7d30\u8a18\u9304", "type": "message"}], "text": "\u904b\u52d5\u985e\u578b", "thumbnailImageUrl": "SPECIFY_YOUR_IMAGE_URL", "title": "\u904b\u52d5\u6642\u9593"}], "type": "carousel"}, "type": "template"}


In [6]:
# 告知handler，當用戶收到文字消息時，作下列辦法
@handler.add(MessageEvent, message=TextMessage)

def replyText(event):
    if '查詢我的歷史運動紀錄' == event.message.text:
        line_bot_api.reply_message(event.reply_token,carousel_template_message)

In [ ]:
# 6.聯絡我們

In [7]:

jsonString = """
{
  "type": "text",
  "text": "                 專案團隊\n-------------------------------------------------\n曾韋翔\n                 0316ff8@gmail.com\n陳子恩\n                 0316ff8@gmail.com\n林尚輝\n                 0316ff8@gmail.com\n徐右儒\n                 0316ff8@gmail.com\n陳正龍\n                 0316ff8@gmail.com\n范綱欣\n                 0316ff8@gmail.com"
}
"""

import json
jsonDict = json.loads(jsonString, strict=False)

print(jsonDict)

{'type': 'text', 'text': '                 專案團隊\n-------------------------------------------------\n曾韋翔\n                 0316ff8@gmail.com\n陳子恩\n                 0316ff8@gmail.com\n林尚輝\n                 0316ff8@gmail.com\n徐右儒\n                 0316ff8@gmail.com\n陳正龍\n                 0316ff8@gmail.com\n范綱欣\n                 0316ff8@gmail.com'}


In [8]:
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)


# 將先前轉入的jsonDict 放置進此方法內
jsonDictToTextSendMessage =  TextSendMessage.new_from_json_dict(jsonDict)
print(jsonDictToTextSendMessage)

{"text": "                 \u5c08\u6848\u5718\u968a\n-------------------------------------------------\n\u66fe\u97cb\u7fd4\n                 0316ff8@gmail.com\n\u9673\u5b50\u6069\n                 0316ff8@gmail.com\n\u6797\u5c1a\u8f1d\n                 0316ff8@gmail.com\n\u5f90\u53f3\u5112\n                 0316ff8@gmail.com\n\u9673\u6b63\u9f8d\n                 0316ff8@gmail.com\n\u8303\u7db1\u6b23\n                 0316ff8@gmail.com", "type": "text"}


In [9]:
# 告知handler，當用戶收到文字消息時，作下列辦法
@handler.add(MessageEvent, message=TextMessage)
def replyText(event):
    if '聯絡我們' == event.message.text:
        line_bot_api.reply_message(event.reply_token,jsonDictToTextSendMessage)


In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.21.0.3 - - [26/Oct/2019 11:15:24] "GET /素材/關注/big-running.jpg HTTP/1.1" 200 -
172.21.0.3 - - [26/Oct/2019 11:15:26] "POST / HTTP/1.1" 200 -
172.21.0.3 - - [26/Oct/2019 11:15:29] "POST / HTTP/1.1" 200 -
172.21.0.3 - - [26/Oct/2019 11:15:36] "POST / HTTP/1.1" 200 -
172.21.0.3 - - [26/Oct/2019 11:15:47] "POST / HTTP/1.1" 200 -
172.21.0.3 - - [26/Oct/2019 11:57:53] "POST / HTTP/1.1" 200 -


In [ ]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])